# Extract OSM road-network detailed

Code for extracting and storing the simplified and detailed OSM road-network

In [1]:
%reload_kedro
%config IPCompleter.use_jedi = False
import osmnx as ox
import networkx as nx
import pandas as pd

2022-03-04 13:00:38,944 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-03-04 13:00:38,965 - kedro.framework.session.session - WARNING - Unable to git describe /Users/ejwillemse/dev/waste_labs_dev/project_rdi_cpt


fatal: Needed a single revision


2022-03-04 13:00:38,986 - root - INFO - ** Kedro project Demand estimation and waste collection routing optimisation for the City of Cape Town
2022-03-04 13:00:38,987 - root - INFO - Defined global variable `context`, `session` and `catalog`
2022-03-04 13:00:38,997 - root - INFO - Registered line magic `run_viz`


In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('data/collection_diagnostics/'))
sys.path.insert(0, os.path.abspath('data/OSM_processing/'))
#rom utils import df_visual, df_prep, gdf_helpers
#import OSM_processing.process_gdf as process_gdf

The following custom filter works for Singapore, given that its road network is quite porous. 

It seems to return non-highway boundaries and entities, which is a problem.

In [3]:
city = 'City of Cape Town'

In [4]:
drive_service_all = '["area"!~"yes"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|corridor|elevator|escalator|proposed|construction|bridleway|abandoned|platform|raceway"]["motor_vehicle"!~"no"]["motorcar"!~"no"]'
exclude_emergency_services = '["service"!~"emergency_access"]'
exclude_custom_services = '["access"!~"private"]["service"!~"parking|parking_aisle|private|emergency_access"]'
exclude_parking_private_emergency = '["access"!~"private"]["service"!~"parking|private|emergency_access"]'
exclude_private = '["access"!~"private"]["service"!~"private"]'
custom_filter_SG = drive_service_all + exclude_emergency_services

It relies too much on `["motor_vehicle"!~"no"]["motorcar"!~"no"]`. Easiest way to filter roads seem to be to use high-way.

In [5]:
custom_filter_SG

'["area"!~"yes"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|corridor|elevator|escalator|proposed|construction|bridleway|abandoned|platform|raceway"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"emergency_access"]'

In [6]:
gap_zones = catalog.load("gap_zones")
gap_zones_sample = gap_zones.loc[gap_zones["OBJECTID"].isin([24645])]
geometry = gap_zones_sample.geometry.values[0]

2022-03-04 13:00:40,533 - kedro.io.data_catalog - INFO - Loading data from `gap_zones` (GeoJSONDataSet)...


In [7]:
%%time
G_full = ox.graph_from_polygon(geometry, custom_filter=custom_filter_SG, simplify=False)

CPU times: user 18.5 s, sys: 506 ms, total: 19 s
Wall time: 19.5 s


In [8]:
nx.gpickle.write_gpickle(G_full, 'temp/data/OSM/SG_full_road.gpickle')

<class 'networkx.utils.decorators.argmap'> compilation 17:5: DeprecationWarning: write_gpickle is deprecated and will be removed in 3.0.Use ``pickle.dump(G, path, protocol)``


In [47]:
node_list_full, edge_list_full = ox.graph_to_gdfs(G_full)
node_list_full = node_list_full.reset_index()
node_list_full_xy = node_list_full.to_crs('EPSG:3414').reset_index()
edge_list_full_xy = edge_list_full.to_crs('EPSG:3414').reset_index()

In [46]:
node_list_full['index'] = node_list_full["osmid"]
node_list_full_xy['index']  = node_list_full_xy["osmid"]

KeyError: 'osmid'

In [30]:
node_list_full.to_csv('temp/data/OSM/SG_full_road_nodes.csv', index=False)
edge_list_full.to_csv('temp/data/OSM/SG_full_road_edges.csv', index=False)
node_list_full_xy.to_csv('temp/data/OSM/SG_full_road_nodes_xy.csv', index=False)
edge_list_full_xy.to_csv('temp/data/OSM/SG_full_road_edges_xy.csv', index=False)

In [40]:
edge_list_full

,osmid,lanes,name,highway,maxspeed,width,oneway,length,ref,access,bridge,tunnel,service,landuse,junction,geometry
0,4351529,2.0,Meadow Road,residential,60.0,5.5,False,7.376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (18.4715332 -33.9559211, 18.4716114..."
1,150916462,4.0,Main Road,secondary,60.0,NaN,False,10.053,M4,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (18.4715332 -33.9559211, 18.4715082..."
2,150916482,3.0,Woolsack Drive,secondary,60.0,NaN,False,10.336,M89,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (18.4715332 -33.9559211, 18.471423 ..."
3,212775078,2.0,Main Road,secondary,60.0,NaN,False,9.770,M4,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (18.4715332 -33.9559211, 18.4715576..."
4,152382551,2.0,Main Road,secondary,60.0,NaN,True,15.180,M4,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (18.4646338 -33.9770134, 18.4646169..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67608,4913464,NaN,Hugon Road,residential,NaN,NaN,False,34.563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (18.4615007 -33.9825025, 18.4613311..."
67609,1035589268,NaN,NaN,service,NaN,NaN,False,3.984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (18.4615007 -33.9825025, 18.4615401..."
67610,1035589267,NaN,NaN,NaN,NaN,NaN,False,24.789,NaN,private,NaN,NaN,NaN,NaN,NaN,"LINESTRING (18.4615401 -33.9824878, 18.4614261..."
67611,1035589267,NaN,NaN,NaN,NaN,NaN,False,1.733,NaN,private,NaN,NaN,NaN,NaN,NaN,"LINESTRING (18.4615401 -33.9824878, 18.4615481..."


In [12]:
%%time
G_simplified = ox.graph_from_polygon(geometry, custom_filter=custom_filter_SG, simplify=True)

CPU times: user 16.2 s, sys: 328 ms, total: 16.6 s
Wall time: 17 s


In [13]:
nx.gpickle.write_gpickle(G_simplified, 'temp/data/OSM/SG_full_road_simple.gpickle')

<class 'networkx.utils.decorators.argmap'> compilation 17:5: DeprecationWarning: write_gpickle is deprecated and will be removed in 3.0.Use ``pickle.dump(G, path, protocol)``


In [14]:
node_list, edge_list = ox.graph_to_gdfs(G_simplified)
node_list_xy = node_list.to_crs('EPSG:3414')
edge_list_xy = edge_list.to_crs('EPSG:3414')

In [15]:
node_list['index'] = node_list.index
node_list_xy['index']  = node_list_xy.index

In [18]:
node_list.to_csv('temp/data/OSM/SG_full_road_simple_nodes.csv', index=False)
edge_list.to_csv('temp/data/OSM/SG_full_road_simple_edges.csv', index=False)
node_list_xy.to_csv('temp/data/OSM/SG_full_road_simple_nodes_xy.csv', index=False)
edge_list_xy.to_csv('temp/data/OSM/SG_full_road_simple_edges_xy.csv', index=False)

## Remove non-highways and unatached nodes

The full network include some boundaries that should be removed. It seems to be any arc without a highway tag.

In [38]:
def filter_highway(nodes, edges):
    print(nodes.shape, edges.shape)
    edges = edges.copy()
    nodes = nodes.copy()
    nodes['u'] = nodes['index']
    edges = edges.loc[~edges['highway'].isna()]
    nodes = nodes.loc[(nodes['u'].isin(edges['u'])) | (nodes['u'].isin(edges['v']))]
    print(nodes.shape, edges.shape)
    return nodes, edges

In [39]:
node_list_full = pd.read_csv('temp/data/OSM/SG_full_road_nodes.csv')
node_list_full.set_index('index', inplace=True)
edge_list_full = pd.read_csv('temp/data/OSM/SG_full_road_edges.csv')
node_list_full_xy = pd.read_csv('temp/data/OSM/SG_full_road_nodes_xy.csv')
node_list_full_xy.set_index('index', inplace=True)
edge_list_full_xy = pd.read_csv('temp/data/OSM/SG_full_road_edges_xy.csv')

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [34]:
nodes_list_full, edges_list_full = filter_highway(node_list_full, edge_list_full)
node_list_full_xy, edge_list_full_xy = filter_highway(node_list_full_xy, edge_list_full_xy)

(33665, 6) (67613, 16)


KeyError: 'index'

In [ ]:
node_list_full.to_csv('data/OSM/SG_full_road_nodes.csv', index=False)
edge_list_full.to_csv('data/OSM/SG_full_road_edges.csv', index=False)
node_list_full_xy.to_csv('data/OSM/SG_full_road_nodes_xy.csv', index=False)
edge_list_full_xy.to_csv('data/OSM/SG_full_road_edges_xy.csv', index=False)

In [ ]:
nodes_list_full = gdf_helpers.create_gdf(node_list_full)
edge_list_full = gdf_helpers.create_gdf(edge_list_full)
G_full_red = ox.graph_from_gdfs(node_list_full, edge_list_full)

In [ ]:
len([node for node in G_full_red.nodes if type(node) is not dict])

In [ ]:
nx.gpickle.write_gpickle(G_full_red, 'data/OSM/SG_full_road_red.gpickle')

In [ ]:
node_list = pd.read_csv('data/OSM/SG_full_road_simple_nodes.csv')
node_list.set_index('u', inplace=True)
edge_list = pd.read_csv('data/OSM/SG_full_road_simple_edges.csv')
node_list_xy = pd.read_csv('data/OSM/SG_full_road_simple_nodes_xy.csv')
node_list_xy.set_index('u', inplace=True)
edge_list_xy = pd.read_csv('data/OSM/SG_full_road_simple_edges_xy.csv')

In [ ]:
node_list, edge_list = filter_highway(node_list, edge_list)
node_list_xy, edge_list_xy = filter_highway(node_list_xy, edge_list_xy)

In [ ]:
node_list.to_csv('data/OSM/SG_full_road_simple_nodes.csv', index=False)
edge_list.to_csv('data/OSM/SG_full_road_simple_edges.csv', index=False)
node_list_xy.to_csv('data/OSM/SG_full_road_simple_nodes_xy.csv', index=False)
edge_list_xy.to_csv('data/OSM/SG_full_road_simple_edges_xy.csv', index=False)

In [ ]:
node_list = gdf_helpers.create_gdf(node_list)
edge_list = gdf_helpers.create_gdf(edge_list)
G_simplified_red = ox.graph_from_gdfs(node_list, edge_list)

In [ ]:
nx.gpickle.write_gpickle(G_simplified_red, 'data/OSM/SG_full_road_simple_red.gpickle')

### Convert to json

In [ ]:
edge_list_full = pd.read_csv('data/OSM/SG_full_road_simple_edges.csv')

In [ ]:
edge_list_reduced = edge_list_full[['geometry', 'length', 'u', 'v', 'key']]
edge_list_reduced = gdf_helpers.create_gdf(edge_list_reduced)

In [ ]:
edge_list_reduced.to_file('data/OSM/SG_full_road_simple_edges.geojson', driver='GeoJSON')

### Simplified road-segment

In [ ]:
edge_list_xy.shape

In [ ]:
edge_list_xy_direct = process_gdf.process_edges(edge_list_xy)
edge_list_xy_direct = gdf_helpers.create_gdf(edge_list_xy_direct, crs='EPSG:3414')
edge_list_xy_direct = edge_list_xy_direct.drop_duplicates(['geom_id_order'])
edge_list_xy_direct.shape

In [ ]:
edge_list.shape

In [ ]:
edge_list_direct = process_gdf.process_edges(edge_list)
edge_list_direct = gdf_helpers.create_gdf(edge_list_direct, crs='EPSG:4326')
edge_list_direct = edge_list_direct.drop_duplicates(['geom_id_order'])
edge_list_direct.shape

In [ ]:
edge_list_direct.to_csv('data/OSM/SG_full_road_simple_edges_bidirect.csv', index=False)
edge_list_xy_direct.to_csv('data/OSM/SG_full_road_simple_edges_xy_bidirect.csv', index=False)